# Calculating rise/set times with astropy

Coordinate system definitions:

| Parameter | Definition | 
|-----------|------------|
| $\alpha$  | Right ascension |
| $\delta$    | Declination |
| $\theta$ | Local sidereal time (LST) | 
| $\theta_0$ | LST in Greenwich | 
| $H$ | Local hour angle | 
| $\phi$ | Observer latitude | 
| $L$ | Observer longitude | 
| $h$ | Object altitude | 
| $A$ | Object azimuth | 

From Meeus' *Astronomical Algorithms* Equation 13.6, 

$$ \sin(h) = \sin(\phi) \sin(\delta) + \cos(\phi) \cos(\delta) \cos(H)$$

and since 

$$ H = \theta - \alpha = \theta_0 - L - \alpha,$$

$$ \sin(h) = \sin(\phi) \sin(\delta) + \cos(\phi) \cos(\delta) \cos(\theta_0 - L - \alpha)$$

In [6]:
%matplotlib inline
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import numpy as np
import matplotlib.pyplot as plt
import datetime
import astropy.units as u
from astropy.coordinates import Angle, EarthLocation, Latitude, \
                                Longitude, get_sun
from astropy.time import Time

##################################################
# IERS table patch from @eteq ####################
from astropy.utils.data import download_file
from astropy.utils import iers
iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, 
                                                      cache=True))
##################################################

# Some convenience functions

def get_altitude(location, RA, dec, LST):
    '''
    Compute altitude of an object given its `RA` and `Dec`
    and an observer's `location` (EarthLocation) and local
    sidereal time `LST`.
    '''
    alt = np.arcsin(np.sin(location.latitude.radian)*np.sin(dec) + 
                    np.cos(location.latitude.radian)*np.cos(dec) *
                    np.cos(LST - RA.radian))
    return alt

def timebracket(t0, dt, N):
    '''
    Generate `N` times bracketing `t0` going
    backwards and forwards `dt` hours
    
    t0 : datetime.datetime
        Initial time
    dt : float
        Number of hours before and after t0
    N : int
        Number of datetimes in range
    '''
    previous_dt = [t0 + datetime.timedelta(hours=i) 
            for i in np.linspace(-dt, 0, N)]
    next_dt = [t0 + datetime.timedelta(hours=i) 
            for i in np.linspace(0, dt, N)]
    return previous_dt, next_dt

def zero_cross_ba(t, a):
    '''
    Find time `t` when values in array `a` go from
    negative to positive (exclude endpoints)
    '''
    condition = (a[:-1] < 0) * (a[1:] > 0)
    return t[condition]

def zero_cross_ab(t, a):
    '''
    Find time `t` when values in array `a` go from
    positive to negative (exclude endpoints)
    '''
    condition = (a[:-1] > 0) * (a[1:] < 0)
    return t[condition]

# Input observer location at Subaru
latitude = '19:49:42.600'
longitude = '-155:28:48.900'
elevation = 0*u.m
location = EarthLocation(lat=latitude, lon=longitude,
                         height=elevation)

LSToption = 'mean' # Choose 'mean' or 'apparent', 
# i.e., accounting for precession only, or also for nutation.

# Set up a range of dates, calculate LST
time = datetime.datetime(2015,5,29)
previous_dates, next_dates = timebracket(time, 24.5, 3000)
previous_times = Time(previous_dates, location=location)
next_times = Time(next_dates, location=location)
previous_LST = previous_times.sidereal_time(LSToption).radian
next_LST = next_times.sidereal_time(LSToption).radian

# Calculate sun's position at present, earlier, later times
sun = get_sun(Time(time, location=location))
previous_alt_sun = get_altitude(location, sun.ra, 
                                sun.dec, previous_LST)
next_alt_sun = get_altitude(location, sun.ra, 
                            sun.dec, next_LST)

# Use time of horizon-crossing to compute sunrise/sunset times
print("previous sunrise:",zero_cross_ba(previous_times, previous_alt_sun)[0])
print("previous sunset:",zero_cross_ab(previous_times, previous_alt_sun)[0])
print("next sunrise:",zero_cross_ba(next_times, next_alt_sun)[0])
print("next sunset:",zero_cross_ab(next_times, next_alt_sun)[0])

previous sunrise: 2015-05-28 15:46:53.737913
previous sunset: 2015-05-28 04:53:59.879960
next sunrise: 2015-05-29 15:43:04.461487
next sunset: 2015-05-29 04:50:10.603535


### Compare to PyEphem

In [7]:
import ephem
sun = ephem.Sun()
obs = ephem.Observer()
obs.lat = latitude
obs.lon = longitude
obs.date = time
obs.pressure = 0 # Turn off pressure/refraction
sun.compute(obs)
print("previous sunrise:", obs.previous_rising(sun, use_center=True))
print("previous sunset:", obs.previous_setting(sun, use_center=True))
print("next sunrise:", obs.next_rising(sun, use_center=True))
print("next sunset:", obs.next_setting(sun, use_center=True))

previous sunrise: 2015/5/28 15:46:34
previous sunset: 2015/5/28 04:51:36
next sunrise: 2015/5/29 15:46:26
next sunset: 2015/5/29 04:52:00


### Results

The $N$ parameter in `timebracket()` controls the density of time points tested for rise/set, and should therefore control the precision of the rise/set times. I'm unsure how PyEphem handles precession/nutation (handled by the IERS tables), so I'm unsure how close the numbers should be. 

Also note that these calculations are for sunrise/sunset as defined by when the solar centroid crosses the horizon, not the crossing of the nearest limb.

| Event | astropy $N = 1000$| astropy $N=3000$ | PyEphem |
|-------|-------------------|------------------|---------|
|previous sunrise| 15:45:35 | 15:46:53 | 15:46:34 |
|previous sunset| 04:53:43 | 04:53:59 | 04:51:36 |
|next sunrise| 15:41:44 | 15:43:04 | 15:46:26 |
|next sunset| 04:49:52 | 04:50:10 | 04:52:00 |

### Benchmarking

In [10]:
%%timeit 
previous_dates, next_dates = timebracket(time, 24.5, 3000)
previous_times = Time(previous_dates, location=location)
next_times = Time(next_dates, location=location)
previous_LST = previous_times.sidereal_time(LSToption).radian
next_LST = next_times.sidereal_time(LSToption).radian
sun = get_sun(Time(time, location=location))
previous_alt_sun = get_altitude(location, sun.ra, 
                                sun.dec, previous_LST)
sunrise = zero_cross_ba(previous_times, previous_alt_sun)[0]
sunset = zero_cross_ab(previous_times, previous_alt_sun)[0]

10 loops, best of 3: 104 ms per loop
